In [1]:
pip install -q fastapi pyngrok transformers accelerate torch uvicorn

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "prranjal/16-bit-Mistral-7B"

tokenizer_mistral = AutoTokenizer.from_pretrained(model_id)
mistral_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "prranjal/16-bit-Llama3.1-8B"

tokenizer_llama = AutoTokenizer.from_pretrained(model_id)
llama_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
def run_mistral(prompt):

    inputs = tokenizer_mistral(prompt, return_tensors="pt").to(mistral_model.device)

    with torch.no_grad():
        output = mistral_model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            top_p=0.9
        )

    return tokenizer_mistral.decode(output[0], skip_special_tokens=True)


In [4]:
def run_llama(prompt):

    inputs = tokenizer_llama(prompt, return_tensors="pt").to(llama_model.device)

    with torch.no_grad():
        output = llama_model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            top_p=0.9
        )

    return tokenizer_llama.decode(output[0], skip_special_tokens=True)


Fast api server with ngrok

In [5]:
from pyngrok import ngrok

ngrok.set_auth_token("token")


In [6]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="Mental Health")

class PromptRequest(BaseModel):
    prompt: str

@app.post("/submit_mistral")
def generate_text(request: PromptRequest):
    return {"response": run_mistral(request.prompt)}


@app.post("/submit_llama")
def generate_text(request: PromptRequest):
    return {"response": run_llama(request.prompt)}

In [7]:
import nest_asyncio

nest_asyncio.apply()

In [11]:
pip install uvicorn==0.35.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: uvicorn
    Found existing installation: uvicorn 0.38.0
    Uninstalling uvicorn-0.38.0:
      Successfully uninstalled uvicorn-0.38.0


In [8]:
import uvicorn
from pyngrok import ngrok


public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url)

uvicorn.run(app, host="0.0.0.0", port=8000)


🚀 Public URL: NgrokTunnel: "https://roastingly-diathetic-ema.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [14433]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


INFO:     2401:4900:8830:e6c1:358b:51b2:87dd:757a:0 - "POST /submit_mistral HTTP/1.1" 200 OK
INFO:     2401:4900:8830:e6c1:358b:51b2:87dd:757a:0 - "POST /submit_llama HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",